### Data Handling of Graphs

Graphs are collections of nodes and edges. PyG allows us to store graphs using
torch_geometric.data.Data. To create a graph, we need two tensors:
* x contains node features
    * for n nodes and m features, the matrix contains n columns and m rows
* edge_index contains connectivity information
    * for k edges, the matrix contains 2 columns and k rows
    * edges are directional

In [8]:
import torch
from torch_geometric.data import Data

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 1, 1],
                          [1, 0, 2, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [2]:
from torch_geometric.datasets import Planetoid

In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [5]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [6]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [7]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8100


In [8]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])